<a href="https://colab.research.google.com/github/rebeccahe1998/ScrapeingStuff/blob/main/Updated_Count_Call_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import random
import os
import re
import numpy as np
import pandas as pd
!pip install html2text
import html2text
!pip install yfinance
import yfinance as yf

1. Get ticker and its full name required for Motley Fool search url
2. Get all the available transcripts urls for the ticker
3. Count how many analysts of text after the key word by splitting keyword 'Call participants'
4. Generate a Excel file

In [2]:
def full_name(ticker):
    return yf.Ticker(ticker).info['longName'].split()[0].split(',')[0]
full_name('DAVA')

'Endava'

In [3]:

user_agent = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36",
                "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.10 (KHTML, like Gecko) Chrome/7.0.540.0 Safari/534.10",
                "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/534.4 (KHTML, like Gecko) Chrome/6.0.481.0 Safari/534.4",
                "Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-US) AppleWebKit/533.4 (KHTML, like Gecko) Chrome/5.0.375.86 Safari/533.4",
                "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/532.2 (KHTML, like Gecko) Chrome/4.0.223.3 Safari/532.2",
                "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.201.1 Safari/532.0",
                "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/3.0.195.27 Safari/532.0",
                "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/530.5 (KHTML, like Gecko) Chrome/2.0.173.1 Safari/530.5",
                "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/534.10 (KHTML, like Gecko) Chrome/8.0.558.0 Safari/534.10",
                "Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/540.0 (KHTML,like Gecko) Chrome/9.1.0.0 Safari/540.0"]

In [4]:
def headers():
    return {'User-Agent':user_agent[random.randint(0, len(user_agent) - 1)],}
def get_urls(full_name, ticker):
    url_ = []
    res = requests.get(f'https://www.fool.com/quote/nasdaq/{full_name}/{ticker}/',
                   headers=headers(),proxies={'http':'150.129.148.88'})
    # test which exchange(NYSE OR NASDAQ)
    if not res.status_code == 200:
        try:
            res = requests.get(f'https://www.fool.com/quote/nyse/{full_name}/{ticker}/',
                   headers=headers(),proxies={'http':'150.129.148.88'})
        except: print('URL might be wrong')
    soup = BeautifulSoup(res.content,'lxml')
    for i in soup.find_all('a', {'data-id' : 'article-list-hl'}, href=re.compile("transcripts")):
        url_.append('https://www.fool.com'+i.get('href'))
    # remove duplictes links 
    url_list = []
    url_date = []
    for url_i in url_:
        if re.split(r'(q\d+-\d+)',url_i)[1] not in url_date:
            url_date.append(re.split(r'(q\d+-\d+)',url_i)[1])
            url_list.append(url_i)
        else: pass
    return url_date,url_list # return 'q3-2020', and url
#get_urls('ENDAVA','DAVA')

In [5]:
def get_call_participated_analyst(ticker):
    full = full_name(ticker) 
    url_date, url_list = get_urls(full,ticker)
    
    inner_url_analyst_count = {'Date':[], ticker: []}

    for i,(inner_url, inner_date) in enumerate(zip(url_list,url_date)):
        time.sleep(np.random.randint(10))
        try: 
            res = requests.get(inner_url,headers=headers(),proxies={'http':'150.129.148.88'})

            inner_url_analyst_count['Date'].append(inner_date) # append date
        except: print('Requests went wrong')

        soup = BeautifulSoup(res.content,'lxml') 
        text = str(res.text)
        text = html2text.html2text(text)
        for i in ['Call Participants','Call participant']:
            text = text.replace(i, 'Call participants')
        participants = text.split('Call participants')[-1] #split by keyword to get participants
        inner_url_analyst_count[ticker].append(len(re.findall(r'Analyst|Unidentified',participants)))
        
    return inner_url_analyst_count

In [6]:
get_call_participated_analyst('DAVA')

{'DAVA': [9, 6, 8, 4, 5, 6, 5],
 'Date': ['q3-2020',
  'q2-2020',
  'q1-2020',
  'q4-2019',
  'q3-2019',
  'q1-2019',
  'q4-2018']}

In [8]:
writer = pd.ExcelWriter('Earnings_Call_Analyst_Count.xlsx')
def get_df(ticker):
    df = pd.DataFrame.from_dict(get_call_participated_analyst(ticker), orient='index').reindex(['Date',ticker])
    df.columns = df.iloc[0]
    df = df[1:]
    return df

def merge_df(ticker_list):
    df = get_df(ticker_list[0])
    for ticc in ticker_list[1:]:
        to_join = get_df(ticc)
        df = pd.concat([df,to_join])
    return df



In [9]:
# trying the code above
ticker_list = ['VNET', 'DAVA']#, 'FROG', 'LOPE', 'CCMP', 'MSGS', 'SONO', 'DOYU', 'WEN']
joined = merge_df(ticker_list)

In [10]:
joined # looks good

,q3-2020,q2-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018,q2-2018
VNET,9,9,5,6,4,3,2,5,7
DAVA,9,6,8,4,5,NaN,6,5,NaN


In [18]:
df = pd.read_excel('scrape_list.xlsx') # read in the ticker excel file
mylist = df['Ticker'].tolist()
ticker_list = mylist[0:10]
ticker_list


['TWOU',
 'DRVN',
 'VNET',
 'DAVA',
 'FROG',
 'LOPE',
 'CCMP',
 'MSGS',
 'SONO',
 'DOYU']

In [19]:
joined = merge_df(ticker_list)

In [20]:
joined

,q4-2020,q3-2020,q2-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018,q3-2018,q2-2018,q1-2021
TWOU,8,7,6,6,6,7,11,10,12,15,NaN,NaN
DRVN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VNET,NaN,9,9,5,6,4,3,2,5,NaN,7,NaN
DAVA,NaN,9,6,8,4,5,NaN,6,5,NaN,NaN,NaN
FROG,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOPE,4,4,3,3,3,1,2,3,3,NaN,NaN,NaN
CCMP,NaN,NaN,7,6,6,5,3,4,4,NaN,NaN,6
MSGS,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SONO,7,6,7,7,8,5,6,5,NaN,4,NaN,6
DOYU,NaN,NaN,NaN,6,5,6,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
writer = pd.ExcelWriter('Earnings_Call_Analyst_Count.xlsx')
joined.to_excel(writer,header = True, index=True)
writer.save()